In [1]:
import os
import sys
from math import floor

# To Avoid Crashes with a lot of nodes
import torch.multiprocessing
from pytorch_lightning import LightningDataModule
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import transforms
from torchvision.datasets import MNIST
import copy
import random

d:\git\fedstellar-venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.path.append(os.path.join(os.path.dirname('__file__'),'../../../..'))  # Parent directory where is the fedstellar module

In [3]:
from fedstellar import learning

In [4]:
import copy
import random

In [5]:
model_path = 'D:/git/fedstellar-venv/fedstellar/app/models/fedstellar_DFL_20_03_2023_10_58_56/'

In [6]:
import pickle as pk

In [7]:
model_file_0=model_path+'participant_0_model.pk'
model_file_1=model_path+'participant_1_model.pk'

In [8]:
with open(model_file_0, 'rb') as f0:
    model0 = pk.load(f0)

In [9]:
with open(model_file_1, 'rb') as f1:
    model1 = pk.load(f1)

In [25]:
param1 = model0.state_dict()

param2 = model1.state_dict()


In [31]:
for layer in param1:
    p1 = param1[layer]
    p2 = param2[layer]
    print(torch.equal(p1, p2))

True
True
True
True
True
True


In [8]:
def label_flipping(dataset, indices, flippingPersent):
    """
    
    """
    new_dataset = dataset
    targets = new_dataset.targets.detach().clone()
    num_indices = len(indices)
    classes = new_dataset.classes
    class_to_idx = new_dataset.class_to_idx
    class_list = [class_to_idx[i] for i in classes]

    num_flipped = int(flippingPersent*num_indices)
    if num_indices == 0:
        return new_dataset
    if num_flipped > num_indices:
        return new_dataset
    flipped_indice = random.sample(indices, num_flipped)
    
    for i in flipped_indice:
        t = targets[i]
        flipped = torch.tensor(random.sample(class_list, 1)[0])        
        while t == flipped:
            flipped = torch.tensor(random.sample(class_list, 1)[0])
        targets[i] = flipped
    new_dataset.targets = targets
    return new_dataset


In [20]:
from skimage.util import random_noise

In [31]:
def datapoison(dataset, indices, poisoned_persent, poisoned_ratio, noise_type="salt"):
    """
    """
    new_dataset = copy.copy(dataset)
    train_data = new_dataset.data.detach().clone()
    num_indices = len(indices)

    num_poisoned = int(poisoned_persent*num_indices)
    if num_indices == 0:
        return new_dataset
    if num_poisoned > num_indices:
        return new_dataset
    poisoned_indice = random.sample(indices, num_poisoned)
    
    for i in poisoned_indice:
        t = train_data[i]
        if noise_type == "salt":
            poisoned = torch.tensor(random_noise(t, mode=noise_type, amount=poisoned_ratio))
        elif noise_type == "gaussian":
            poisoned = torch.tensor(random_noise(t, mode=noise_type, mean=0, var=poisoned_ratio, clip=True))
        elif noise_type == "speckle":
            poisoned = torch.tensor(random_noise(t, mode=noise_type, mean=0, var=poisoned_ratio, clip=True))
        else:
            print("ERROR: poison attack type not supported.")
            poisoned = t
        train_data[i] = poisoned
    new_dataset.targets = train_data
    return new_dataset

In [2]:
mnist_train = MNIST(f"{sys.path[0]}/data", train=True, download=False, transform=transforms.ToTensor())

In [3]:
rows_by_sub = floor(len(mnist_train) / 3)

In [4]:
trainset = tr_subset = Subset(
            mnist_train, range(0 * rows_by_sub, (0 + 1) * rows_by_sub))

In [5]:
trainset.indices

range(0, 20000)

In [6]:
mnist_train, mnist_val = random_split(
            tr_subset,
            [
                round(len(tr_subset) * (1 - 0.2)),
                round(len(tr_subset) * 0.2),
            ],
        )


In [26]:
a = mnist_train.data[0][0:5]
a

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]], dtype=torch.uint8)

In [52]:
noise_type = "s&p"

In [56]:
poisoned = torch.tensor(random_noise(a, mode=noise_type,amount=0.1 ))
poisoned

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=torch.float64)

In [16]:
class ChangeableSubset(Subset):
    """
    Could change the elements in Subset Class
    """
    def __init__(self, dataset, indices, label_flpping=False, data_change=False, flippingPersent=0, dataChangePersent=0):
        new_dataset = copy.copy(dataset)
        self.dataset = new_dataset
        self.indices = indices
        self.label_flpping = label_flpping
        self.flippingPersent = flippingPersent
        self.data_change = data_change
        self.dataChangePersent = dataChangePersent

        if self.label_flpping == True:
            self.dataset = label_flipping(self.dataset, self.indices, self.flippingPersent)
        # if data_change:
    
    def __getitem__(self, idx):
        if isinstance(idx, list):
            return self.dataset[[self.indices[i] for i in idx]]
        return self.dataset[self.indices[idx]]
    
    def __len__(self):
        return len(self.indices)


In [19]:
mnist_train.data.shape


torch.Size([60000, 28, 28])

In [17]:
tr_subset = ChangeableSubset(
            mnist_train, range(0, 100), label_flpping=False
        )